# Regularizacion


In [38]:
#Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm

from random import random
import math

import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score



In [39]:
#df = pd.read_csv("/Users/normalazo/Desktop/env1/Entrega3_LR/regLin.csv")
#df = pd.read_csv("/Users/normalazo/Desktop/env1/Entrega3_LR/regLin2.csv")
#df = pd.read_csv("/Users/normalazo/Desktop/env1/Entrega3_LR/regLin3.csv")
#df = pd.read_csv("/Users/normalazo/Desktop/env1/Entrega3_LR/regLin4.csv")
df = pd.read_csv("/Users/normalazo/Desktop/env1/Entrega3_LR/regLinPoli.csv")



In [40]:
# This data does not have headers so each attribute or field is simply enumerated
df.describe()

,X,X2,X3,X4,X5,y
count,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000
mean,48.052380,3113.669342,1.542203,-0.011250,628.000000,3113.490528
std,28.379936,2891.956247,0.452374,0.717016,297.191016,2892.963581
min,0.006314,0.000040,-2.199666,-0.999993,114.000000,0.000040
25%,23.174764,537.069692,1.365015,-0.731530,371.000000,522.757565
50%,47.655827,2271.077831,1.678116,-0.046938,628.000000,2262.728789
75%,71.372550,5094.040864,1.853531,0.732296,885.000000,5093.260718
max,99.873062,9974.628611,1.999448,1.000000,1142.000000,10028.067820


In [41]:
# Here df.columns i# Separate both train and test as well as the response variable

# a list of all the columns and df.columns[0:-1] is all columns minus the last which is y. 
# If the data had headers you could use column names: df[['column1','column2','etc']]
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[[df.columns[-1]]], train_size=0.75)

In [42]:
# index for selecting data 0.75 is the percentage in training
index=np.array([1 if random() < 0.75 else 0 for i in range(len(df))])


In [43]:
# Separate both train and test as well as the response variable
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
Y_train=np.asarray(Y_train)
Y_test=np.asarray(Y_test)

#plt.scatter(X_test, Y_test,  color='black')

In [44]:
# Normalizar no ayuda mucho pero sale igual al de sklearn. Para que las alturas del pdf signifiquen lo mismo 
from sklearn import preprocessing

scalerX = preprocessing.StandardScaler()
scalerY = preprocessing.StandardScaler()

scalerX.fit(X_train)
X_train=scalerX.transform(X_train)


scalerY.fit(Y_train)
Y_train=scalerY.transform(Y_train)

X_test=scalerX.transform(X_test)
Y_test=scalerY.transform(Y_test)


#plt.scatter(X_test, Y_test,  color='black')

In [45]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [46]:
# Train the model using the training sets
regr.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [47]:
def ILR(X_train, Y_train, learningC, w, lam ):
    numrows = len(X_train)    # 3 rows in your example
    numcols = len(X_train[0]) # 2 columns in your example

    for j in range (0,numrows):

        x = np.ones(1)
        x = np.concatenate((x,X_train[j]), axis=0)

        #calculate the value of 
        res = np.dot(w,x)

        #difference 
        dif = Y_train[j] - res

        #print dif

        for i in range (0,numcols +1 ):
            if(i==0):
                w[i] = w[i]+dif*learningC*x[i]
            else:
                w[i] = w[i]+dif*learningC*x[i]-w[i]*lam
            
    return w


In [48]:
#Numero de Columnas y Renglones
numrows = len(X_train)    # 3 rows in your example
numcols = len(X_train[0]) # 2 columns in your example

#inicializacion de vector de w's
w = np.ones(numcols+1)


In [49]:
for j in range (0,5):
    w = ILR(X_train, Y_train, .001, w, 0.0)
    X_mod = np.c_[ X_test, np.ones(X_test.shape[0]) ]
    Y_pred = np.dot(w,X_mod.T)
    

In [50]:
#MSE
mean_squared_error(Y_test, Y_pred)

0.19831553743740082

In [51]:
#Vector de Pesos
w

array([ 0.02270489,  0.34915581,  0.57345319,  0.07516063,  0.03778597,
        0.02530873])

# Regularizacion con Mini Batch

In [52]:
def ILR_batch(X_train, Y_train, learningC, w, lam , numBatch, batchSize):
    numrows = len(X_train)    # 3 rows in your example
    numcols = len(X_train[0]) # 2 columns in your example
    x_barra = np.zeros(numcols+1)
    dif_acum = 0
    

    for j in range (0,numBatch):
        
        idx = np.random.randint(X_train.shape[0], size=batchSize)
        idx
        X_Batch = X_train[idx,:]
        
        
        for k in range (0,batchSize):
            x = np.ones(1)
            x = np.concatenate((x,X_train[k]), axis=0)
            x_barra = x_barra + x
            
            #calculate the value of 
            res = np.dot(w,x)
            
            #difference 
            dif = Y_train[k] - res
            dif_acum = dif_acum + dif
        
        x_barra=x_barra/batchSize
        dif_acum=dif_acum/batchSize

        #actualizamos cada peso
        for i in range (0,numcols +1 ):
            if(i==0):
                w[i] = w[i]+dif_acum*learningC*x_barra[i]
            else:
                w[i] = w[i]+dif_acum*learningC*x_barra[i]-w[i]*lam

    return w


In [53]:
#inicializacion de vector de w's
w = np.ones(numcols+1)

for j in range (0,5):
    w = ILR_batch(X_train, Y_train, .001, w, 0.01 ,2000,50)
    X_mod = np.c_[ X_test, np.ones(X_test.shape[0]) ]
    Y_pred = np.dot(w,X_mod.T)

In [54]:
#MSE
mean_squared_error(Y_test, Y_pred)

0.91662137672417676

In [55]:
#Vector de Pesos
w

array([  5.05618001e-02,  -5.01041518e-08,  -1.61463782e-07,
         2.82108793e-07,  -1.80845372e-07,  -4.02040521e-07])